# According to Figure 11: Expected modularity according to the different community strengths
## ACP

In [ ]:
from Expected_mod import Trans_C1, APWP
import Expected_mod as ex 
import load_graph
import importlib
import bz2
import os 
import pandas as pd 
import json
import time
import networkx as nx 
importlib.reload(load_graph)
path='mcp_acp_data//k10_l10//result//acpc//'
path2='datasets//'
filelists=[i for i in os.listdir('mcp_acp_data//k10_l10//result//acpc')]
filelists.sort()
T=[]
clustering=[]
value=[]
for file in filelists:
    print(file)
    graph=file[:-9]+'.txt'
    g=load_graph.read_g(path2+graph)
    edge=[]
    p=[]
    for u,v,w in g.edges(data=True):
        edge.append((u,v))
        p.append(w['weight'])

    with bz2.open(path+file,'rt') as f:
        data=json.load(f)
    # convert the table to dataframe
    clustering_df=pd.DataFrame(data['tables']['clustering'])
 
    print(Trans_C1(list(clustering_df['clabel'])),len(Trans_C1(list(clustering_df['clabel']))))
    print(list(clustering_df['probability']))

    cluster=Trans_C1(list(clustering_df['clabel']))
    clustering.append(cluster)
    Emod=ex.APWP(edge,p,cluster)
    print('----------graph: ',graph,'----------')
    print('cluster:',cluster)
    print('-----------Ex modularity',Emod,'-----------')
    value.append(Emod)

    

# calculate AMI according to Figure 12: AMI score according to the community strengths

In [ ]:
from Expected_mod import Trans_C2
from sklearn.metrics.cluster import adjusted_mutual_info_score as ami

standard_c=[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
node=100
AMI=[]
for c in clustering:
    c=Trans_C2(c,node)
    
    score=ami(standard_c,c)
    print(score)
    AMI.append(score)
print(AMI)


# calculate AMI according toFigure 13: AMI score according to the increasing number of clusters, community strength s = 0.7

In [ ]:

from Expected_mod import Trans_C1, APWP
import Expected_mod as ex 
import load_graph
import importlib
import time
import networkx as nx 
from Expected_mod import Trans_C2
from sklearn.metrics.cluster import adjusted_mutual_info_score as ami 

importlib.reload(load_graph)
path='mcp_acp_data//l10_p0.3//results//acpc_results//'
path2='mcp_acp_data//l10_p0.3//datasets//'
filelists=[i for i in os.listdir('mcp_acp_data//l10_p0.3//results//acpc_results//')]
filelists.sort()
print(filelists)
T=[]
clustering=[]
X_=[]
l=10
NMI=[]
for file in filelists:
    print(file)
    graph=file[:-9]+'.txt'
    g=load_graph.read_g(path2+graph)
   

    with bz2.open(path+file,'rt') as f:
        data=json.load(f)
    # convert the table to dataframe
    clustering_df=pd.DataFrame(data['tables']['clustering'])

    print(Trans_C1(list(clustering_df['clabel'])),len(Trans_C1(list(clustering_df['clabel']))))
    print(list(clustering_df['probability']))

    cluster=Trans_C1(list(clustering_df['clabel']))
    strr=[]
    flag=0
   
    for _ in graph:
        if _=='k':
            
            flag=1
            continue
        if flag==0:
            continue
        if _<'0' or _>'9':
           
            break
        strr.append(_)
    
    k=sum([int(strr[i])*(10**(len(strr)-(i+1))) for i in range(len(strr))])
    c=[i for i in range(k*l)]
    
    stad_cluster=[c[x:x+l] for x in range(0, len(c), l)]
    nmi_=ami(Trans_C2(stad_cluster,k*l),Trans_C2(cluster,k*l))
    NMI.append(nmi_)
    X_.append(k)

# Experiment to low prob. according to Performances under Different Probability Distributions (RQ5)

In [ ]:
from Expected_mod import Trans_C1, APWP
import Expected_mod as ex 
import load_graph
import importlib
import time
import networkx as nx 
import pandas as pd 
import json
import bz2
import os
from tkinter import Tcl
from os import listdir
importlib.reload(load_graph)
acpc_clustering={}
path1='mcp_acp_data//l10_p0.3_evolving_lowP1//result//acpc//'
path2='mcp_acp_data//l10_p0.3_evolving_lowP1//'
filelists=[i for i in os.listdir(path1)]
filelists=Tcl().call('lsort','-dict',filelists)
T=[]
clustering=[]
value=[]
for file in filelists:
   
    file_list=file.split('_')
    k=[i for i in file_list if i[0]=='k'][0][1:]
    
    graph=file[:-9]+'.txt'
    print('file',file)
    print(graph)
    g=load_graph.read_g(path2+graph)
    edge=[]
    p=[]
    for u,v,w in g.edges(data=True):
        edge.append((u,v))
        p.append(w['weight'])

    with bz2.open(path1+file,'rt') as f:
        data=json.load(f)
    # convert the table to dataframe
    clustering_df=pd.DataFrame(data['tables']['clustering'])
   

    cluster=Trans_C1(list(clustering_df['clabel']))
    acpc_clustering.update({k:cluster})
    print(cluster)
   
print(acpc_clustering)
  

## save results

In [ ]:

import json
path='mcp_acp_data//l10_p0.3_evolving_lowP1//result//acpc//'
with open(path+'acpc_increaseK_lowP.json','w') as fp:
          json.dump(acpc_clustering,fp,indent=4) # With indent=4 (pretty format)

In [ ]:
with open(path+'acpc_increaseK_lowP.json','r') as fp:
    data=json.load(fp)
print(data)

# Experiment of high prob. according to Performances under Different Probability Distribu- tions (RQ5)

## have completely same graph structures with low prob.

In [ ]:
from Expected_mod import Trans_C1, APWP
import Expected_mod as ex 
import load_graph
import importlib
import time
import networkx as nx 
import pandas as pd 
import json
import bz2
import os
from tkinter import Tcl
from os import listdir
importlib.reload(load_graph)
acpc_clustering={}
path1='mcp_acp_data//l10_p0.3_evolving_highP1//result//acpc//'
path2='mcp_acp_data//l10_p0.3_evolving_highP1//'
filelists=[i for i in os.listdir(path1)]
filelists=Tcl().call('lsort','-dict',filelists)
T=[]
clustering=[]
value=[]
for file in filelists:
   
    file_list=file.split('_')
    k=[i for i in file_list if i[0]=='k'][0][1:]
    
    graph=file[:-9]+'.txt'
    print('file',file)
    print(graph)
    g=load_graph.read_g(path2+graph)
    edge=[]
    p=[]
    for u,v,w in g.edges(data=True):
        edge.append((u,v))
        p.append(w['weight'])

    with bz2.open(path1+file,'rt') as f:
        data=json.load(f)
    # convert the table to dataframe
    clustering_df=pd.DataFrame(data['tables']['clustering'])
   

    cluster=Trans_C1(list(clustering_df['clabel']))
    acpc_clustering.update({k:cluster})
    print(cluster)
   
print(acpc_clustering)


## saving results

In [ ]:
import json
path='mcp_acp_data//l10_p0.3_evolving_highP1//result//acpc//'
with open(path+'acpc_increaseK_highP.json','w') as fp:
          json.dump(acpc_clustering,fp,indent=4) # With indent=4 (pretty format)

In [ ]:
with open(path+'acpc_increaseK_highP.json','r') as fp:
    data=json.load(fp)
print(data)

# Experiment polarized graph according to Performances under Different Probability Distributions (RQ5)

In [ ]:
from Expected_mod import Trans_C1, APWP
import Expected_mod as ex 
import load_graph
import importlib
import time
import networkx as nx 
import pandas as pd 
import json
import bz2
import os
from tkinter import Tcl
from os import listdir
importlib.reload(load_graph)
acpc_clustering={}
path1='mcp_acp_data//l50_k2_p0.18_polarized_graph//result//acpc//'
path2='mcp_acp_data//l50_k2_p0.18_polarized_graph//'
filelists=[i for i in os.listdir(path1) if i !='.DS_Store']
filelists=Tcl().call('lsort','-dict',filelists)
T=[]
clustering=[]
value=[]
for file in filelists:
    print(file)
    file_list=file.split('_')
    k=[i for i in file_list if i[0]=='k'][0][1:]
    name=file_list[-2]
    
    graph=file[:-9]+'.txt'
    print('file',file)
    print(graph)
    g=load_graph.read_g(path2+graph)
    edge=[]
    p=[]
    for u,v,w in g.edges(data=True):
        edge.append((u,v))
        p.append(w['weight'])

    with bz2.open(path1+file,'rt') as f:
        data=json.load(f)
    # convert the table to dataframe
    clustering_df=pd.DataFrame(data['tables']['clustering'])
    

    cluster=Trans_C1(list(clustering_df['clabel']))
    acpc_clustering.update({name:cluster})
    print(cluster)
   
print(acpc_clustering)


In [ ]:
import json
path='mcp_acp_data//l50_k2_p0.18_polarized_graph//result//acpc//'
with open(path+'acpc_polarized.json','w') as fp:
          json.dump(acpc_clustering,fp,indent=4) # With indent=4 (pretty format)

## krogan2006 core & mips

In [ ]:
from Expected_mod import Trans_C1, APWP
import Expected_mod as ex 
import load_graph
import importlib
import time
import networkx as nx 
import pandas as pd 
import json
import bz2
import os
from tkinter import Tcl
from os import listdir
importlib.reload(load_graph)
acpc_clustering={}
path1='mcp_acp_data//krogan2006_core//intersec_mips//acpc_results//'
path2='mcp_acp_data//krogan2006_core//intersec_mips/net//krogan2006_core_mips_net.txt'

T=[]
clustering=[]
value=[]


g=load_graph.read_g(path2)
file1='krogan2006_core_mips_net.json.bz2'

with bz2.open(path1+file1,'rt') as f:
    data=json.load(f)
# convert the table to dataframe
clustering_df=pd.DataFrame(data['tables']['clustering'])

cluster=Trans_C1(list(clustering_df['clabel']))
k=len(cluster)
acpc_clustering.update({k:cluster})
print(cluster)
   
print(acpc_clustering)
   # T.append(t2-t1)
    #print(clustering_df.to_string)
    

In [ ]:

import json
path='mcp_acp_data//krogan2006_core//intersec_mips//acpc_results//'
with open(path+'acpc_k188.json','w') as fp:
          json.dump(acpc_clustering,fp,indent=4) # With indent=4 (pretty format)

## krogan2006 extended & mips

In [ ]:
from Expected_mod import Trans_C1, APWP
import Expected_mod as ex 
import load_graph
import importlib
import time
import networkx as nx 
import pandas as pd 
import json
import bz2
import os
from tkinter import Tcl
from os import listdir
importlib.reload(load_graph)
acpc_clustering={}
path1='mcp_acp_data//krogan2006_extended//intersec_mips//acpc_results//'
path2='mcp_acp_data//krogan2006_extended//intersec_mips//krogan2006_extented_mips_net.txt'

T=[]
clustering=[]
value=[]


g=load_graph.read_g(path2)
file1='krogan2006_extented_mips_net.json.bz2'

with bz2.open(path1+file1,'rt') as f:
    data=json.load(f)
# convert the table to dataframe
clustering_df=pd.DataFrame(data['tables']['clustering'])

cluster=Trans_C1(list(clustering_df['clabel']))
k=len(cluster)
print(cluster)
acpc_clustering.update({k:cluster})
print(cluster)
   
print(acpc_clustering)
   # T.append(t2-t1)
    #print(clustering_df.to_string)
    

In [ ]:

import json
path='mcp_acp_data//krogan2006_extended//intersec_mips//acpc_results//'
with open(path+'acpc_k195.json','w') as fp:
          json.dump(acpc_clustering,fp,indent=4) # With indent=4 (pretty format)

# Effect of Shifting Probabilities (RQ2).

In [ ]:
from Expected_mod import Trans_C1, APWP
import Expected_mod as ex 
import load_graph
import importlib
import time
import networkx as nx 
import pandas as pd 
import json
import bz2
import os
from tkinter import Tcl
from os import listdir
importlib.reload(load_graph)
acpc_clustering={}
path1='mcp_acp_data//l10_k10_p0.4_multiplied//new//acp_results//'
path2='mcp_acp_data//l10_k10_p0.4_multiplied//new//datasets//'
filelists=[i for i in os.listdir(path1) if i !='.DS_Store']
filelists=Tcl().call('lsort','-dict',filelists)
T=[]
clustering=[]
value=[]
print(filelists)
for file in filelists:
    print(file)
    file_list=file.split('_')
    strr=[]
    flag=0
   
    for _ in file:
        if _=='i':
            
            flag=1
            continue
        if flag==0:
            continue
        if _<'0' or _>'9':
           
            break
        strr.append(_)
    print(strr)
    multi=str(0)+'.'+str(strr[1])#sum([int(strr[i])*(10**(len(strr)-(i+1))) for i in range(len(strr))])
    print(multi)
 
    
    graph=file[:-9]+'.txt'
    
    print('g',graph)
    g=load_graph.read_g(path2+graph)
    edge=[]
    p=[]
    for u,v,w in g.edges(data=True):
        edge.append((u,v))
        p.append(w['weight'])

    with bz2.open(path1+file,'rt') as f:
        data=json.load(f)
    # convert the table to dataframe
    clustering_df=pd.DataFrame(data['tables']['clustering'])


    cluster=Trans_C1(list(clustering_df['clabel']))
    acpc_clustering.update({multi:cluster})
    print(cluster)
   
print(acpc_clustering)
 
    

In [ ]:

import json
path='mcp_acp_data//l10_k10_p0.4_multiplied//new//acp_results//'
with open(path+'acpc.json','w') as fp:
          json.dump(acpc_clustering,fp,indent=4) # With indent=4 (pretty format)